In [1]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,572 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,046 kB]
Get:13 http://security.ubuntu.com/

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install graphframes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=84373d16aaaac862d468a025e21aa60e0a079681f03cc0bd76296dc3ccce3989
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 28 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [4]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
spark

In [8]:
ratings_pd = pd.read_csv('/content/drive/MyDrive/CSE 488/CSE488_Lab10_Recommender_Systems/ratings.csv')
ratings_pd.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
ratings = spark.createDataFrame(ratings_pd)
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [10]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [11]:
training.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
|     1|    356|   4.0|964980962|
|     1|    362|   5.0|964982588|
|     1|    367|   4.0|964981710|
+------+-------+------+---------+
only showing top 20 rows



In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [13]:
als = ALS(maxIter = 10, regParam = 0.01, userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop")
model = als.fit(training)

In [14]:
predictions = model.transform(test)
predictions.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   148|   1197|   3.0|1482548478| 3.6056194|
|   148|   4886|   3.0|1482548505| 3.0743728|
|   148|   5618|   3.0|1482548682| 3.3058562|
|   148|  76093|   3.0|1482548702| 3.8225746|
|   148|  79702|   4.0|1482548751| 3.4244547|
|   148|  98243|   4.5|1482548883| 3.6021593|
|   148| 110102|   4.0|1482548669|  5.008391|
|   148| 122886|   3.5|1482548686| 1.4692315|
|   243|     62|   5.0| 837155394| 4.0481043|
|   243|    353|   5.0| 837155394| 2.0429873|
|   243|    376|   3.0| 837155577| 3.0856795|
|   243|    434|   5.0| 837155121| 4.3676033|
|   243|    595|   3.0| 837155088| 4.5345764|
|   243|    648|   5.0| 837155455| 4.1598434|
|    31|      1|   5.0| 850466616| 3.8507066|
|    31|     25|   2.0| 850466616| 2.7368226|
|    31|     62|   4.0| 850466616| 4.7777815|
|    31|    788|   2.0| 850466679| 3.6752195|
|    31|   1057|   5.0| 850466810|

In [15]:
predictions.filter("userId=1").show()

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|   1580|   3.0|964981125| 4.1221447|
|     1|   2366|   4.0|964982462|  4.195129|
|     1|   1127|   4.0|964982513| 4.3757443|
|     1|    804|   4.0|964980499|  2.156644|
|     1|   2542|   5.0|964981265| 4.2934017|
|     1|   3062|   4.0|964981872|  4.545818|
|     1|   2797|   4.0|964981710|  4.648486|
|     1|   1031|   5.0|964982653| 3.6268148|
|     1|   1030|   3.0|964982903|  4.530669|
|     1|    157|   5.0|964984100| 2.7404943|
|     1|     47|   5.0|964983815| 4.4042053|
|     1|   2054|   4.0|964981747|  4.309114|
|     1|   2161|   5.0|964981710|  4.345688|
|     1|    500|   3.0|964981208|  4.221819|
|     1|   2193|   4.0|964981710| 4.8120093|
|     1|   1517|   5.0|964981107|  4.867967|
|     1|   3702|   5.0|964983737| 3.7979748|
|     1|    780|   3.0|964984086|  4.134447|
|     1|   1298|   5.0|964984086| 4.2603416|
|     1|  

In [16]:
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE: " , rmse)


RMSE:  1.1181570882366951


In [17]:
userRecs = model.recommendForAllUsers(3)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3022, 6.3730745...|
|     2|[{4232, 8.645004}...|
|     3|[{2261, 6.6226587...|
|     4|[{4678, 8.32959},...|
|     5|[{4678, 11.704926...|
|     6|[{3494, 6.1707735...|
|     7|[{7720, 10.33854}...|
|     8|[{3040, 9.917869}...|
|     9|[{3030, 7.155734}...|
|    10|[{4167, 6.60327},...|
|    11|[{2693, 7.903845}...|
|    12|[{2693, 9.469198}...|
|    13|[{2693, 7.0894423...|
|    14|[{5480, 9.567629}...|
|    15|[{2565, 10.203494...|
|    16|[{6932, 5.6387978...|
|    17|[{3022, 6.3221245...|
|    18|[{3200, 5.039747}...|
|    19|[{55363, 4.944137...|
|    20|[{535, 7.574398},...|
+------+--------------------+
only showing top 20 rows



In [18]:
user535_np = np.array(userRecs.filter("userId=535").select("recommendations").collect())
print(user535_np)

[[[[3.08800000e+03 9.59358978e+00]
   [8.63200000e+04 9.36163712e+00]
   [2.37200000e+03 9.00205994e+00]]]]


In [19]:
movieRecs = model.recommendForAllItems (3)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|      1|[{99, 6.1422844},...|
|      3|[{542, 7.928565},...|
|      5|[{542, 5.42611}, ...|
|      6|[{99, 6.1705494},...|
|      9|[{126, 8.4243145}...|
|     12|[{147, 9.33026}, ...|
|     13|[{296, 5.961418},...|
|     15|[{502, 8.897667},...|
|     16|[{549, 7.764832},...|
|     17|[{375, 7.733756},...|
|     19|[{197, 6.3133116}...|
|     20|[{399, 5.852331},...|
|     22|[{77, 6.767294}, ...|
|     26|[{423, 6.765664},...|
|     27|[{536, 8.833008},...|
|     28|[{296, 9.099183},...|
|     31|[{329, 7.460198},...|
|     34|[{344, 6.6447744}...|
|     40|[{311, 3.626689},...|
|     41|[{35, 6.820612}, ...|
+-------+--------------------+
only showing top 20 rows



In [20]:
movie1_np = np.array(movieRecs.filter("movieId=1").select("recommendations").collect())
print(movie1_np)

[[[[ 99.           6.14228439]
   [393.           6.07453728]
   [557.           6.03036404]]]]


# Finding the Air craft, country flying mission and target country for those missions in which time on target was less than 1000.

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Assuming you have a SparkSession
spark = SparkSession.builder.appName("MissionAnalysis").getOrCreate()

# Assuming you have a DataFrame named missions_df
# Replace this with your actual DataFrame and column names
# Example DataFrame creation:
data = [
    ("Aircraft1", "Country1", "Target1", 800),
    ("Aircraft2", "Country2", "Target2", 1200),
    # Add more rows as needed
]

columns = ["Aircraft", "Country Flying Mission", "Target Country", "Time on Target"]

missions_df = spark.createDataFrame(data, columns)

# Filter missions where Time on Target is less than 1000
filtered_missions = missions_df.filter(col("Time on Target") < 1000)

# Display the results
filtered_missions.show()


+---------+----------------------+--------------+--------------+
| Aircraft|Country Flying Mission|Target Country|Time on Target|
+---------+----------------------+--------------+--------------+
|Aircraft1|              Country1|       Target1|           800|
+---------+----------------------+--------------+--------------+

